In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [ ]:
data= pd.read_csv("Churn_Modelling.csv")
data.head()

In [ ]:
#  preprocessing the data
data= data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [ ]:
# encode categorical variable
label_encoder_gender= LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])

In [ ]:
data

In [ ]:
#  one hot encoded Geography
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoder= onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder_df= pd.DataFrame(geo_encoder, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [ ]:
geo_encoder_df

In [ ]:
# combine one hot encoded column eith original data
data= pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

In [ ]:
# split the data onto features and target 
X= data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [ ]:
## Split the data in training and tetsing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
# save the encoder and scaler for later use
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo, file)
    
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

#  ANN eegression problem statement

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
# build the model
model= Sequential([
    Dense(64, activation='relu', input_shape=(x.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

#  compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

# moel summary
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# set up Tensorboard
dog_dir = "regressionlogs/fit/"+datetime.datetime.now().strftime("%y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=dog_dir, histogram_freq=1)


In [ ]:
# set up Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir reqressionlogs/fit

In [ ]:
# Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'test mae : {test_mae}')


In [ ]:
model.save('regression_model.h5')